In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [40]:
import torch
import sklearn
import random
# 장치 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 난수 시드 설정
torch.manual_seed(1)
random.seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)
    
# 데이터 로드
train = pd.read_csv("/kaggle/input/2021-ai-midterm-p5/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-midterm-p5/test.csv")
submission = pd.read_csv("/kaggle/input/2021-ai-midterm-p5/submit_sample.csv")

#데이터 확인
print(train.head())
print(test.head())
print(submission.head())

# 데이터 전처리
# 필요 없는 데이터 삭제 
# InternetService와 [OlineSecurity ~ StreamingMovies] 까지 연관이 있다고 생각하여 해당 컬럼 삭제
train = train.drop(['index','Unnamed: 0','customerID','OnlineSecurity','OnlineBackup',
                    'DeviceProtection','TechSupport','StreamingTV','StreamingMovies'],axis=1)
test = test.drop(['index','Unnamed: 0','customerID','OnlineSecurity','OnlineBackup',
                    'DeviceProtection','TechSupport','StreamingTV','StreamingMovies'],axis=1)


from sklearn.preprocessing import StandardScaler, LabelEncoder,MinMaxScaler


# 난측값 채우기
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
# x_train = imputer.fit_transform(x_train)
# x_test = imputer.transform(x_test)

# data 실수화 
le = LabelEncoder()
train['Churn'] = le.fit_transform(train['Churn'])
#gender 처리
train['gender'] = le.fit_transform(train['gender'])
test['gender'] = le.transform(test['gender'])
#Partner 처리
train['Partner'] = le.fit_transform(train['Partner'])
test['Partner'] = le.transform(test['Partner'])
#Dependents 처리
train['Dependents'] = le.fit_transform(train['Dependents'])
test['Dependents'] = le.transform(test['Dependents'])
#PhoneService 처리
train['PhoneService'] = le.fit_transform(train['PhoneService'])
test['PhoneService'] = le.transform(test['PhoneService'])
#MultipleLines 처리
train['MultipleLines'] = le.fit_transform(train['MultipleLines'])
test['MultipleLines'] = le.transform(test['MultipleLines'])
#InternetService 처리
train['InternetService'] = le.fit_transform(train['InternetService'])
test['InternetService'] = le.transform(test['InternetService'])
#Contract 처리
train['Contract'] = le.fit_transform(train['Contract'])
test['Contract'] = le.transform(test['Contract'])



#PaperlessBilling 처리
train['PaperlessBilling'] = le.fit_transform(train['PaperlessBilling'])
test['PaperlessBilling'] = le.transform(test['PaperlessBilling'])
train = train.drop(['PaperlessBilling'],axis=1)
test = test.drop(['PaperlessBilling'],axis=1)
#PaymentMethod 처리
train['PaymentMethod'] = le.fit_transform(train['PaymentMethod'])
test['PaymentMethod'] = le.transform(test['PaymentMethod'])
train = train.drop(['PaymentMethod'],axis=1)
test = test.drop(['PaymentMethod'],axis=1)

#TotalCharge 처리
# train['TotalCharges'] = train['TotalCharges'].astype(float)
# test['TotalCharges'] = test['TotalCharges'].astype(float)
train = train.drop(['TotalCharges'],axis=1)
test = test.drop(['TotalCharges'],axis=1)
# 배열로 변환

x_train = np.array(train.drop(['Churn'],axis=1))
y_train = np.array(train['Churn'])
x_test = np.array(test)

#데이터 확인
print(train.head())
print(test.head())
print(x_train)

# 데이터 변형(scale)
scaler = StandardScaler()
# scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)



# 데이터 텐서에 올리기
x_train = torch.Tensor(x_train).to(device)
x_test = torch.Tensor(x_test).to(device)
y_train = torch.Tensor(y_train).to(device)

# 데이터 셋 
from torch.utils.data import DataLoader,TensorDataset
train_dataset = TensorDataset(x_train,y_train)

# 데이터 모양 확인
print(x_train.shape , y_train.shape)

# Layer 설정
layer1 = torch.nn.Linear(10,64,bias =True).to(device)
layer2 = torch.nn.Linear(64,10,bias =True).to(device)
layer3 = torch.nn.Linear(10,1,bias =True).to(device)
relu = torch.nn.ReLU().to(device)
dropout = torch.nn.Dropout(0.3).to(device)

# Layer 초기화
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)
torch.nn.init.xavier_normal_(layer3.weight)


# Model 설정
model = torch.nn.Sequential(layer1,relu,dropout,
                            layer2,relu,dropout,
                            layer3).to(device)

# 학습 파라미터 설정
epochs = 1000
lr = 0.01
batch_size = 100

# loss 함수 이진 분류
# loss = torch.nn.CrossEntropyLoss()
# loss = torch.nn.MSELoss()
loss =torch.nn.BCELoss()

# optim 
optim = torch.optim.Adam(model.parameters(),lr=lr)

#data loader
data_loader = DataLoader(dataset=train_dataset,batch_size=batch_size)

# 학습
model.train()
for epoch in range(epochs):
    optim.zero_grad()
    h = model(x_train)
    cost = loss(torch.sigmoid(h),y_train.unsqueeze(1))
    cost.backward()
    optim.step()
    if epoch % 100 == 0:
        print(epoch , cost.item())

In [41]:
with torch.no_grad():
    model.eval()
    predict = torch.sigmoid(model(x_test))
predict = predict >= torch.Tensor([0.5]).to(device)
submission['Churn'] = predict.int()
submission.to_csv("submission.csv",index=False)